In [1]:
import pandas as pd
from utils import util
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
def plot_video_mse(pred_csv_path, video_name):
    pred_df = pd.read_csv(pred_csv_path)
    pred_df['video_name'], pred_df['frame id'] = zip(*pred_df['img_path'].map(util.get_video_name_and_frame_num))
    pred_df = pred_df[pred_df['video_name'] == video_name]
    pred_df = pred_df.reset_index(drop=True)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=pred_df['frame id'], y=pred_df['mse'], mode='lines+markers', name='mse'))
    # smoothing
    window = 10
    fig.add_trace(go.Scatter(x=pred_df['frame id'], y=pred_df['mse'].rolling(window).mean(), mode='lines', name=f'smoothed mse(window={window})'))
    
    # add a horizontal "target" line
    emo = 0
    for i in pred_df['frame id']:
        _emo = pred_df[pred_df['frame id'] == i]['emotion'].values[0]
        if _emo != emo or i == 0:
            if _emo == 0:
                color = 'blue'
            elif _emo == 1:
                color = 'red'
            else:
                color = 'green'

            fig.add_shape(
                type="line",
                x0=i,
                y0=0,
                x1=i,
                y1=pred_df['mse'].max(),
                line=dict(
                    color=color,
                    width=2,
                    dash="dot",
                ),
            )
            emo = _emo 
    
    fig.update_layout(
        title=video_name,
        xaxis_title="frame id",
        yaxis_title="mse",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f"
        )
    )
    
    fig.show()

In [3]:
def plot_allvideo_mse_histgram(pred_csv_path, except_video_name_list=None):
    pred_df = pd.read_csv(pred_csv_path)
    pred_df['video_name'], pred_df['frame id'] = zip(*pred_df['img_path'].map(util.get_video_name_and_frame_num))
    if except_video_name_list is not None:
        pred_df = pred_df[~pred_df['video_name'].isin(except_video_name_list)]
    pred_df = pred_df.reset_index(drop=True)
    fig = go.Figure()
    
    # histogram
    for i , emo in enumerate(['other', 'comfort', 'discomfort']):
        fig.add_trace(go.Histogram(x=pred_df[pred_df['emotion'] == i]['mse'], name=emo))
    
    fig.update_layout(
        title="mse stacked histgram",
        xaxis_title="mse",
        yaxis_title="count",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f"
        ),
        barmode='stack',
        bargap=0.1,
        bargroupgap=0.1
    )
    
    fig.show()
    
    return pred_df

In [8]:
pred_csv_path = '/mnt/iot-qnap3/mochida/medical-care/emotionestimation/reports/PIMD_A/5_d_a/epoch20/fold2/pred.csv'

In [9]:
video_name = 'video19'
plot_video_mse(pred_csv_path, video_name)

In [10]:
except_video_name_list = ['video' + str(i) for i in range(18, 26)]
except_video_name_list.remove(video_name)
print(except_video_name_list)
pred_df = plot_allvideo_mse_histgram(pred_csv_path, except_video_name_list)

['video18', 'video20', 'video21', 'video22', 'video23', 'video24', 'video25']


In [140]:
# except pred_df 90% quantile
top_df = pred_df[pred_df['mse'] < pred_df['mse'].quantile(0.1)]
top_df = top_df.sort_values('img_path')
top_df = top_df.reset_index(drop=True)
top_df.to_csv('../../temp/bottom10_AU_fold0.csv', index=False)
top_df.head(10)

,img_path,emotion,mse,video_name,frame id
0,/mnt/iot-qnap3/mochida/medical-care/emotionest...,0,0.050122,video10,462
1,/mnt/iot-qnap3/mochida/medical-care/emotionest...,0,0.049925,video10,484
2,/mnt/iot-qnap3/mochida/medical-care/emotionest...,0,0.048077,video10,551
3,/mnt/iot-qnap3/mochida/medical-care/emotionest...,0,0.047358,video10,552
4,/mnt/iot-qnap3/mochida/medical-care/emotionest...,0,0.047427,video10,553
5,/mnt/iot-qnap3/mochida/medical-care/emotionest...,0,0.048517,video10,554
6,/mnt/iot-qnap3/mochida/medical-care/emotionest...,0,0.047010,video10,555
7,/mnt/iot-qnap3/mochida/medical-care/emotionest...,0,0.048622,video10,556
8,/mnt/iot-qnap3/mochida/medical-care/emotionest...,0,0.048406,video10,557
9,/mnt/iot-qnap3/mochida/medical-care/emotionest...,0,0.049487,video10,558
